In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
enviroment_name = 'CartPole-v1'
env = gym.make(enviroment_name)

In [4]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode} score {score}")

Episode 0 score 17.0
Episode 1 score 19.0
Episode 2 score 13.0
Episode 3 score 12.0
Episode 4 score 27.0


In [5]:
env.close()

## Train

In [6]:
log_path = os.path.join('Training', 'Logs')
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [7]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 205  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 241         |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008709147 |
|    clip_fraction        | 0.0933      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00633    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.01        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 55          |
-----------------------------------------
---

## Saving / Loading

In [8]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
# model.save(PPO_path)
# model = PPO.load(PPO_path, env=env)

## Validation

In [9]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(492.7, 21.9)

In [10]:
env.close()

## Test

In [11]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
        print('info', info)
        break

info [{'terminal_observation': array([ 2.4195263 ,  0.9121439 , -0.12843165, -0.3900652 ], dtype=float32)}]


In [12]:
env.close()

## Logs

In [13]:
training_log_path = os.path.join(log_path, 'PPO_1')
!tensorboard --logdir={training_log_path}

^C


## Callbacks

In [32]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
save_path = os.path.join('Training', 'Saved Models')

In [38]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=475, verbose=1) # 475 = max reward
eval_callback = EvalCallback(
    env,
    callback_on_new_best=stop_callback,
    eval_freq=10000,
    best_model_save_path=save_path,
    verbose=1
)

In [34]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [35]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_10
-----------------------------
| time/              |      |
|    fps             | 422  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 351         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008197522 |
|    clip_fraction        | 0.0968      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00293    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.78        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0162     |
|    value_loss           | 54.1        |
-----------------------------------------
--

## Changing Policies

In [39]:
net_arch = dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128]) # eq =[128,128]
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cuda device


In [40]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_12
-----------------------------
| time/              |      |
|    fps             | 376  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 314         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014157834 |
|    clip_fraction        | 0.211       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | 0.00106     |
|    learning_rate        | 0.0003      |
|    loss                 | 4.65        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0247     |
|    value_loss           | 20.4        |
-----------------------------------------
--

## Alternate algorythm